In [ ]:
file = 'INF2178_A1_data.xlsx'

import pandas as pd
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the first sheet in the Excel file
data = pd.read_excel(file, sheet_name=0)

# Checking for missing values by variables
missing_values = data.isnull().sum()
print(missing_values)

#  Calculating occupancy Rate
# For rooms
data['OCCUPANCY_ROOM_RATE'] = data['OCCUPIED_ROOMS'] / data['CAPACITY_ACTUAL_ROOM']
# For beds
data['OCCUPANCY_BED_RATE'] = data['OCCUPIED_BEDS'] / data['CAPACITY_ACTUAL_BED']
print("Average of OCCUPANCY ROOM RATE", data['OCCUPANCY_ROOM_RATE'].mean())
print("Average of OCCUPANCY BED RATE", data['OCCUPANCY_BED_RATE'].mean())

# define a general Welch's t-test function
# where dataset is the dataframe; categories is string of different category types; cont_y is number of continuour variables
# return is dictionary contains all the comparison of the t-test results
def t_test(dataset, categories, cont_y):
    category_type = data[categories].unique()
    results = {}
    for i in range(len(category_type)):
        for j in range(i+1, len(category_type)):
            category1 = category_type[i]
            category2 = category_type[j]
            sample1 = dataset.loc[dataset[categories] == category1, cont_y]
            sample2 = dataset.loc[dataset[categories] == category2, cont_y]
            t_stat, p_val = stats.ttest_ind(sample1, sample2, equal_var=False)
            results[f'{category1} , {category2}'] = (t_stat, p_val)

    # Output the results
    for key, (value1, value2) in results.items():
            print(f'{key}: t-statistic = {value1}, p-value = {value2}')


t_test(data, 'CAPACITY_TYPE', 'SERVICE_USER_COUNT')
t_test(data, 'SECTOR', 'SERVICE_USER_COUNT')

# define a general statistics function
def general_stats(dataset, cont_y):
    mean = np.round(np.mean(dataset[cont_y]), 2)
    median = np.round(np.median(dataset[cont_y]), 2)
    mini = np.round(dataset[cont_y].min(), 2)
    maxi = np.round(dataset[cont_y].max(), 2)
    std_dev = np.round(dataset[cont_y].std(), 2)
    print (f'Mean: {mean}')
    print (f'Mediam: {median}')
    print (f'Min: {mini}')
    print (f'Max: {maxi}')
    print (f'SD: {std_dev}')
general_stats(data, 'SERVICE_USER_COUNT')

#visulize the data for more detailed analysis
#historgram

# setting up the grid
plt.figure(figsize=(12, 7))

# histogram of service user count
plt.subplot(1, 2, 1)
sns.histplot(data['SERVICE_USER_COUNT'], bins=30,kde=True)
plt.title('Distribution of Service User Count')
plt.axvline(np.mean(data['SERVICE_USER_COUNT']), color='red', linestyle='--')
plt.axvline(np.median(data['SERVICE_USER_COUNT']), color='green', linestyle='-')

# boxplot by sector
plt.subplot(1, 2, 2)
sns.boxplot(x='SECTOR', y='SERVICE_USER_COUNT', data=data)
plt.title('Service User Count by Sector')

plt.show()

# calculate the sum of each group in program model
count1 = (data['PROGRAM_MODEL'] == 'Emergency').sum()
count2 = (data['PROGRAM_MODEL'] == 'Transitional').sum()
print (count1, count2)

# boxplot by program model
plt.figure(figsize=(12, 7))
sns.boxplot(x='PROGRAM_MODEL', y='SERVICE_USER_COUNT', data=data)
plt.title('Service User Count by Program Model')

plt.show()

# the trend of service user count
plt.figure(figsize=(12, 7))
sns.lineplot(x='OCCUPANCY_DATE', y='SERVICE_USER_COUNT', data=data)
plt.title('Service User Count Trend')

plt.show()

